In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import nltk

In [2]:
# Create an empty list to store the preprocessed data = []

NewText_data=[]

# Open the text file for reading
with open('human_chat.txt', 'r', encoding='utf-8') as file:
    # Read each line of the text file
    for line in file:
        # Remove leading/trailing whitespaces and newline characters
        line = line.strip()
        
        # Convert the line to lowercase
        line = line.lower()
        
        # Append the line to the preprocessed_data list
        NewText_data.append(line)


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
x = vectorizer.fit_transform(NewText_data)

In [4]:
# Create empty lists to store asks and responses
asks = []
responses = []


In [5]:
# Iterate over the preprocessed data
for i in range(len(NewText_data)-1):
    # Assign the current line as the ask and the next line as the response
    ask = NewText_data[i]
    response = NewText_data[i+1]
    
    # Append the ask and response to their respective lists
    asks.append(ask)
    responses.append(response)


In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test=train_test_split(x,test_size=0.2,random_state=42)

In [7]:
x_train.shape

(1196, 2734)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Read conversation data from a text file
conversation_data = []
with open('human_chat.txt', 'r', encoding='utf-8') as file:
    for line in file:
        conversation_data.append(line.rstrip())

# Extract unique response categories
response_categories = set()
for line in conversation_data:
    if line.startswith("Human"):
        response = line.split(": ")[1]
        response_categories.add(response)

print("Unique response categories:", response_categories)

# Tokenize the conversation data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(conversation_data)

# Convert conversation data to sequences
sequences = tokenizer.texts_to_sequences(conversation_data)

# Pad sequences to have the same length
max_seq_length = 2800
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length)

num_features = len(tokenizer.word_index) + 1
num_classes = len(response_categories)

# Split the data into input and target sequences
input_sequences = padded_sequences[:-1]
target_sequences = padded_sequences[1:]

# Convert target sequences to one-hot encoded format
target_sequences_one_hot = np.zeros((len(target_sequences), max_seq_length, num_features), dtype=np.float32)
for i, sequence in enumerate(target_sequences):
    for j, index in enumerate(sequence):
        target_sequences_one_hot[i, j, index] = 1

# Split the data into training and validation sets
split = int(0.8 * len(input_sequences))
x_train = input_sequences[:split]
y_train = target_sequences_one_hot[:split]
x_val = input_sequences[split:]
y_val = target_sequences_one_hot[split:]

# Reshape the input data to 3 dimensions
x_train = np.expand_dims(x_train, axis=2)
x_val = np.expand_dims(x_val, axis=2)

# Define the model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(max_seq_length, num_features)))
model.add(Dense(num_features, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

# Example input text
input_text = "Hello, how are you?"

# Convert input text to a sequence
input_sequence = tokenizer.texts_to_sequences([input_text])
input_sequence = pad_sequences(input_sequence, maxlen=max_seq_length)

# Generate responses using the trained model
predicted_response = model.predict(input_sequence)
predicted_response_text = tokenizer.sequences_to_texts(predicted_response.argmax(axis=-1))

print("Input text:", input_text)
print("Predicted response:", predicted_response_text)





Unique response categories: {"Brooooo!!! It's so important. When I got to California beaches I have to carry 3 litres of lotion for the whole day.", 'I went to the local cafe and had a double espresso. It was delicious. What about you?', "Thats pretty cool! I'm a sofa tester in cupertino.", "Does not having had electricity help you empathize more with people of differing backgrounds and/or makes you feel a bit a distance from those who don't understand?", "how's living in Japan?", "I recently tried it for the first time and you're right - it had a wonderful melted chocolate bar taste! I tried it up in a place in San Francisco and they had delicious homemade marshmallows too!", "I'm great, thanks. I just ate a delicious breakfast, which always sets the day up right.", 'oh wow, what was the last half marathon you did?', 'Woah! That is fascinating! I will have to pick up a copy of the book to read.', "Biology. I'm gonna be a doc someday ha", 'Oooo the Mandalorian?!?!', 'kpop...', "yeah...

MemoryError: Unable to allocate 43.8 GiB for an array with shape (1494, 2800, 2809) and data type float32